In [5]:
from typing import Tuple, List
import os
import pickle
from enum import Enum
import nbimporter
from Intergral import rect_ii_sum, rect_sii_sum, integral_image, integral_of_squares
import numpy as np

class FeatureType(Enum):
    TWO_H = 1       # two horizontal rectangles
    TWO_V = 2       # two vertical rectangles
    THREE_H = 3     # three horizontal rectangles
    THREE_V = 4     # three vertical rectangles
    FOUR = 5        # four rectangles (2x2 pattern)

# Feature represented as tuple: (type, x, y, w, h)
def generate_features(img_w=24, img_h=24, step=2, cache_path="./cache/features_cache.pkl") -> List[Tuple]:

    os.makedirs(os.path.dirname(cache_path), exist_ok=True)
    
    if os.path.exists(cache_path):
        print(f"[INFO] Loading cached features from '{cache_path}'...")
        with open(cache_path, "rb") as f:
            features = pickle.load(f)
        return features

    print("[INFO] Generating Haar features...")
    features = []
    for w in range(1, img_w+1, step):
        for h in range(1, img_h+1, step):
            for x in range(0, img_w - w + 1, step):
                for y in range(0, img_h - h + 1, step):
                    if x + 2*w <= img_w:
                        features.append((FeatureType.TWO_H, x, y, w, h))
                    if y + 2*h <= img_h:
                        features.append((FeatureType.TWO_V, x, y, w, h))
                    if x + 3*w <= img_w:
                        features.append((FeatureType.THREE_H, x, y, w, h))
                    if y + 3*h <= img_h:
                        features.append((FeatureType.THREE_V, x, y, w, h))
                    if x + 2*w <= img_w and y + 2*h <= img_h:
                        features.append((FeatureType.FOUR, x, y, w, h))

    print(f"[INFO] Generated {len(features)} features. Saving to '{cache_path}'...")
    with open(cache_path, "wb") as f:
        pickle.dump(features, f)
        
    return features

# Caculate value of the Haar feature include standard deviation normalization
def eval_feature(ii, sii, feature, offset=(0,0)):
    """
    ii, sii: integral images of whole image
    feature: (t, fx, fy, fw, fh) where fx,fy,fw,fh are in base window coordinates (e.g. 24x24)
    offset: (offx, offy) top-left coordinate of the detection window in the original image
    """
    t, fx, fy, fw, fh = feature
    offx, offy = offset

    # convert feature local coords to global coords in the image
    x = fx + offx
    y = fy + offy
    w = fw
    h = fh

    area = w * h
    # helper to compute sums quickly
    if t == FeatureType.TWO_H:
        white_ii = rect_ii_sum(ii, x, y, w, h)
        black_ii = rect_ii_sum(ii, x + w, y, w, h)
        white_sii = rect_sii_sum(sii, x, y, w, h)
        black_sii = rect_sii_sum(sii, x + w, y, w, h)

        sum_ii = white_ii + black_ii
        sum_sii = white_sii + black_sii
        mean = sum_ii / area
        variance = (sum_sii / area) - (mean * mean)
        if variance > 0:
            stdDev = np.sqrt(variance)
            return (white_ii - black_ii) / stdDev
        else:
            return (white_ii - black_ii)

    if t == FeatureType.TWO_V:
        white_ii = rect_ii_sum(ii, x, y, w, h)
        black_ii = rect_ii_sum(ii, x, y + h, w, h)
        white_sii = rect_sii_sum(sii, x, y, w, h)
        black_sii = rect_sii_sum(sii, x, y + h, w, h)
        sum_ii = white_ii + black_ii
        sum_sii = white_sii + black_sii
        mean = sum_ii / area
        variance = (sum_sii / area) - (mean * mean)
        if variance > 0:
            stdDev = np.sqrt(variance)
            return (white_ii - black_ii) / stdDev
        else:
            return (white_ii - black_ii)

    if t == FeatureType.THREE_H:
        a_ii = rect_ii_sum(ii, x, y, w, h)
        b_ii = rect_ii_sum(ii, x + w, y, w, h)
        c_ii = rect_ii_sum(ii, x + 2*w, y, w, h)

        a_sii = rect_sii_sum(sii, x, y, w, h)
        b_sii = rect_sii_sum(sii, x + w, y, w, h)
        c_sii = rect_sii_sum(sii, x + 2*w, y, w, h)

        sum_ii = a_ii + b_ii + c_ii
        sum_sii = a_sii + b_sii + c_sii
        mean = sum_ii / area
        variance = (sum_sii / area) - (mean * mean)
        if variance > 0:
            stdDev = np.sqrt(variance)
            return (a_ii + c_ii - b_ii) / stdDev
        else:
            return (a_ii + c_ii - b_ii)

    if t == FeatureType.THREE_V:
        a_ii = rect_ii_sum(ii, x, y, w, h)
        b_ii = rect_ii_sum(ii, x, y + h, w, h)
        c_ii = rect_ii_sum(ii, x, y + 2*h, w, h)

        a_sii = rect_sii_sum(sii, x, y, w, h)
        b_sii = rect_sii_sum(sii, x, y + h, w, h)
        c_sii = rect_sii_sum(sii, x, y + 2*h, w, h)

        sum_ii = a_ii + b_ii + c_ii
        sum_sii = a_sii + b_sii + c_sii
        mean = sum_ii / area
        variance = (sum_sii / area) - (mean * mean)
        if variance > 0:
            stdDev = np.sqrt(variance)
            return (a_ii + c_ii - b_ii) / stdDev
        else:
            return (a_ii + c_ii - b_ii)

    if t == FeatureType.FOUR:
        a_ii = rect_ii_sum(ii, x, y, w, h)
        b_ii = rect_ii_sum(ii, x + w, y, w, h)
        c_ii = rect_ii_sum(ii, x, y + h, w, h)
        d_ii = rect_ii_sum(ii, x + w, y + h, w, h)

        a_sii = rect_sii_sum(sii, x, y, w, h)
        b_sii = rect_sii_sum(sii, x + w, y, w, h)
        c_sii = rect_sii_sum(sii, x, y + h, w, h)
        d_sii = rect_sii_sum(sii, x + w, y + h, w, h)

        sum_ii = a_ii + b_ii + c_ii + d_ii
        sum_sii = a_sii + b_sii + c_sii + d_sii
        mean = sum_ii / area
        variance = (sum_sii / area) - (mean * mean)
        if variance > 0:
            stdDev = np.sqrt(variance)
            return (a_ii + d_ii - b_ii - c_ii) / stdDev
        else:
            return (a_ii + d_ii - b_ii - c_ii)

    raise ValueError('Unknown feature')